## Import Library

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import cv2

In [4]:
# direktory
train_path = './data/train/'
test_path = './data/test/' 

weight, height = 32, 32
batch_size = 32

train_data_gen = ImageDataGenerator(
    rescale = 1.0/255
)
test_data_gen = ImageDataGenerator(
    rescale = 1.0/255
)

train_dataset = train_data_gen.flow_from_directory(
    train_path,
    target_size = (weight, height),
    batch_size = batch_size,
    class_mode = 'binary',
    shuffle = True
)
test_dataset = test_data_gen.flow_from_directory(
    test_path,
    target_size = (weight, height),
    batch_size = batch_size,
    class_mode = 'binary',
    shuffle = False
)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [5]:
# check kelas
train_dataset.class_indices

{'FAKE': 0, 'REAL': 1}

In [6]:
train_dataset.image_shape

(32, 32, 3)

## CNN Model

In [7]:
# mendefinisikan model
model = models.Sequential()

# convolutional layers
model.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape=(weight, height, 3) ))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten layer
model.add(layers.Flatten())

# dense layers
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# summary model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 2, 128)        0

In [8]:
# save best model
weight_path = '../code/'

model_checkpoint = ModelCheckpoint(
    filepath = weight_path,
    save_best_only = True,
    save_weights_only = True,
    mode = 'max',
    monitor = 'val_accuracy'
)

## Run Model

In [10]:
history = model.fit(train_dataset,
                    epochs = 2,
                    validation_data = test_dataset,
                    callbacks = [model_checkpoint])

Epoch 1/2
3125/3125 [==============================] - 1682s 537ms/step - loss: 0.3366 - accuracy: 0.8547 - val_loss: 0.2454 - val_accuracy: 0.9008
Epoch 2/2
3125/3125 [==============================] - 423s 135ms/step - loss: 0.2319 - accuracy: 0.9070 - val_loss: 0.2192 - val_accuracy: 0.9128


In [11]:
# load best weigh to model
model.load_weights(weight_path)

# buat prediksi
prediksi = model.predict(test_dataset)

625/625 [==============================] - 40s 63ms/step


In [12]:
binary_predictions = (prediksi > 0.5).astype(int)
binary_predictions

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [0]])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# true label
true_labels = test_dataset.classes

# buat confussion matrix
sns.heatmap(confusion_matrix(true_labels, binary_predictions), annot=True)

# print report classification
classification_report(true_labels, binary_predictions)

plt.title("Confusion Matrix")
plt.xlabel("Prediksi")
plt.ylabel("Actual Classes")
plt.show()

In [4]:
# test upscaling
from PIL import Image

# Baca citra dengan Pillow
image = Image.open('./data/test/FAKE/1.jpg')

# Tentukan faktor penskalaan (misalnya, 4 kali lipat)
scale_factor = 10

# Upscale citra menggunakan Pillow
upscaled_image = image.resize((image.width * scale_factor, image.height * scale_factor), Image.ANTIALIAS)

# Simpan citra hasil upscaling
upscaled_image.save('./upscaled_image.jpeg')


C:\Users\ariop\AppData\Local\Temp\ipykernel_17984\3289309780.py:11: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  upscaled_image = image.resize((image.width * scale_factor, image.height * scale_factor), Image.ANTIALIAS)


In [3]:
import zipfile

def extract_zip_file(file_path, extract_path):
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

# Usage
# Provide the path to the zip file and the path where it should be extracted
extract_zip_file('./data/dataset.zip', './data/')


In [ ]:
import zipfile

# Open the zip file in read mode
with zipfile.ZipFile('./data/dataset.zip', 'r') as zip_ref:
    # Loop through each file
    for file in zip_ref.namelist():
        # Open each file
        with zip_ref.open(file) as f:
            # Read the file
            print(f.read())
